\# **Notebook for reproducing CCAM results**

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

2.1.0+cu121
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install scipy==1.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 31.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
jax 0.4.20 requires scipy>=1.9, but you have scipy 1.8.1 which is incompatible.
jaxlib 0.4.20+cuda12.cudnn89 requires scipy>=1.9, but you have scipy 1.8.1 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch_geometric.datasets import Planetoid, Amazon,WebKB,FB15k_237,HeterophilousGraphDataset,WikipediaNetwork,CitationFull,Actor
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.utils import homophily,add_self_loops, is_undirected,to_networkx,from_networkx,to_undirected, to_dense_adj, dense_to_sparse
import time


An example with the Texas dataset

In [ ]:

#dataset = Planetoid(root='data/Planetoid', name='cora', transform=NormalizeFeatures())
#dataset = Amazon(root='data/amazon', name='Computers', transform=NormalizeFeatures())
dataset = WebKB(root='data/WebKB', name='texas',transform = NormalizeFeatures())
#dataset = CitationFull(root='data/Citeseer', name='Citeseer', to_undirected = False)#
#dataset = WikipediaNetwork(root='data/WikipediaNetwork', name='squirrel')
#dataset = HeterophilousGraphDataset(root='data/minesweeper', name='minesweeper')
#dataset = Actor(root='data/Actor')

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.
data.y
print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')


print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

data_und = to_undirected(data.edge_index)
data_und,_= add_self_loops(data_und, num_nodes=data.num_nodes)
data.x.shape# soit un objet py_torch geometric


Dataset: texas():
Number of graphs: 1
Number of features: 1703
Number of classes: 5

Data(x=[183, 1703], edge_index=[2, 325], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])
Number of nodes: 183
Number of edges: 325
Average node degree: 1.78
Number of training nodes: 870
Training node label rate: 4.75
Has isolated nodes: False
Has self-loops: True
Is undirected: False


torch.Size([183, 1703])

To test on other datasets that don't come from py torch geometric, you need :
data_und, the adjacency matrix, must be a tensor of size [2, number of edges].
data.x, the features matrix, is a tensor of size [Number of nodes, num_features].
dataset.num_classes, the number of classes for classification
The easiest way is to create a py torch geometric dataset directly with these elements
https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Data.html#torch_geometric.data.Data



Package for Ollivier and Forman Curvature.


In [ ]:
!pip install GraphRicciCurvature
from GraphRicciCurvature.OllivierRicci import OllivierRicci
import networkx as nx
# load GraphRicciCuravture package
from GraphRicciCurvature.OllivierRicci import OllivierRicci
from GraphRicciCurvature.FormanRicci import FormanRicci


import networkx as nx
import numpy as np
import math
%matplotlib inline
import matplotlib.pyplot as plt

# to print logs in jupyter notebook
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

# load GraphRicciCuravture package
from GraphRicciCurvature.OllivierRicci import OllivierRicci
from GraphRicciCurvature.FormanRicci import FormanRicci

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 22.4 MB/s eta 0:00:00


Compute Curvature Matrix

In [ ]:
def mean_courbure(G,curvature="formanCurvature") : #formanCurvature/ricciCurvature
    total_forman_curvature = 0
    nombre_d_elements = 0
    for sommet, voisins in G_curv.adj.items():
        for voisin, attributs in voisins.items():
            forman_curvature = attributs.get(curvature, None)
            if forman_curvature is not None:
                total_forman_curvature += forman_curvature
                nombre_d_elements += 1
    if nombre_d_elements > 0:
        moyenne_forman_curvature = total_forman_curvature / nombre_d_elements
    return moyenne_forman_curvature

In [ ]:
A = torch.squeeze(to_dense_adj(data_und)).numpy()
G=nx.Graph(A)
start_time = time.time()
#curvature = "ricciCurvature" #Ollivier Curvature
#orc = OllivierRicci(G, alpha=0.5, verbose="TRACE")#Ollivier Curvature
curvature = "formanCurvature" #Coubure de forman
orc = FormanRicci(G)
start_time = time.time()
orc.compute_ricci_curvature()
G_curv = orc.G.copy()
print(time.time() - start_time)


mat_ricci=np.zeros((data.num_nodes,data.num_nodes))
mat_riccim=np.zeros((data.num_nodes,data.num_nodes))
#mat_ricci_ent=np.zeros((data.num_nodes,data.num_nodes))

mean_mr = mean_courbure(G)


for a,i in enumerate(np.unique(G_curv)):
    for j in list(G_curv.neighbors(a)):
        if G_curv[i][j][curvature]>=mean_mr :
            mat_ricci[a][j]=G_curv[i][j][curvature]
        if G_curv[i][j][curvature]<=mean_mr :
            mat_riccim[a][j]=G_curv[i][j][curvature]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Filtering the adjancence matrix according to curvature

In [ ]:
curvp = torch.Tensor(mat_ricci)
curvm = torch.Tensor(mat_riccim)
curvp = curvp.nonzero().t().contiguous()
curvm = curvm.nonzero().t().contiguous()

edge_index_curvp,_ = add_self_loops(curvp, num_nodes=data.num_nodes)
edge_index_curvm,_ = add_self_loops(curvm, num_nodes=data.num_nodes)

edge_index_curvp = to_undirected(edge_index_curvp)
edge_index_curvm = to_undirected(edge_index_curvm)
is_undirected(edge_index_curvm)

True

Measurement of homophily on edges presented by Zhu et al. and the measurement Curvature-Constrained homophily

In [ ]:
print(homophily(data_und, data.y))
print(homophily(edge_index_curvp, data.y))
print(homophily(edge_index_curvm, data.y))


0.30779391527175903
0.4072657823562622
0.47826087474823


Possibly building a two-hop based on curvature.

In [ ]:
from torch_geometric.utils import add_self_loops
num_nodes = data.num_nodes
new_edge_index = []

def build_two_hop(mat_adj):

    for node in range(num_nodes):
        neighbors = mat_adj[1][mat_adj[0] == node]

        for neighbor in neighbors:
            neighbors_of_neighbor = mat_adj[1][mat_adj[0] == neighbor]

            neighbors_of_neighbor = neighbors_of_neighbor[neighbors_of_neighbor != node]

            for neighbor_of_neighbor in neighbors_of_neighbor:
                new_edge_index.append([node, neighbor_of_neighbor.item()])

    two_hop_tensor = torch.tensor(new_edge_index, dtype=torch.long).t()
    two_hop_tensor,_ = add_self_loops(two_hop_tensor, num_nodes=data.num_nodes)
    return two_hop_tensor

two_hop_tensor = build_two_hop(edge_index_curvm)#edge_index_curvm/curvp => matrice d'adjance négative/positive
print(homophily(two_hop_tensor, data.y))
is_undirected(two_hop_tensor)
two_hop_tensor.shape

0.5602510571479797


torch.Size([2, 11311])

GCN

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x.to(device), edge_index.to(device))
        x = x.relu()
        x = F.dropout(x.to(device), p=0.5, training=self.training)
        x = self.conv2(x.to(device), edge_index.to(device))
        return x.to(device)

5

GAT

In [ ]:
from torch_geometric.nn import GATConv
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GATConv(dataset.num_features, hidden_channels,heads=8,dropout= 0.5)
        self.conv2 = GATConv(hidden_channels*8, dataset.num_classes,heads=1,dropout= 0.5)

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x.to(device), edge_index.to(device)))
        x = F.dropout(x.to(device), p=0.5, training=self.training)
        x = self.conv2(x.to(device), edge_index.to(device))
        return x.to(device)


In [ ]:
criterion = torch.nn.CrossEntropyLoss()

def train(t,v):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x.to(device), two_hop_tensor.to(device)) #Remplacer edge_index_curvm par edge_index_curvp ou two hop tensor selon le choix de matrice d'adjacence.
      loss = criterion(out[data_train_mask], data.y.to(device)[data_train_mask])  # Compute the loss solely based on the training nodes.
      val_loss = criterion(out[v].to(device), data.y.to(device)[v])  # Compute the loss solely based on the val nodes.
      loss.backward() # Derive gradients.
      #val_loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss,val_loss
def test(mask):
      model.eval()
      out = model(data.x.to(device), two_hop_tensor.to(device))#Remplacer edge_index_curvm par edge_index_curvp ou two hop tensor selon le choix de matrice d'adjacence.
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct = pred[mask] == data.y.to(device)[mask]  # Check against ground-truth labels.
      acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return acc

In [ ]:
import time
import numpy as np
ramdom = True
moy = []
ep = []
T = []
p_m =[]
nb_runs = 10
hidden_channels = 32
lr = 0.005
weight_decay = 5e-6
moy = []
ep = []
for runs in range(nb_runs):
    model = GCN(hidden_channels=hidden_channels).to(device)
    #model = GAT(hidden_channels=hidden_channels).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    criterion = torch.nn.CrossEntropyLoss()
    print("runs", runs)
    start_time = time.time()
    shuffled_indices = torch.randperm(len(data.y))
    if ramdom == True :


        indices = [False for i in range(len(data.y))]
        mask_train = [True if i in shuffled_indices[:20*(torch.max(data.y)+1)] else False for i in range(len(indices))]
        mask_val = [True if i in shuffled_indices[20*(torch.max(data.y)+1):20*(torch.max(data.y)+1)+500] else False for i in range(len(indices))]
        mask_test = [True if i in shuffled_indices[20*(torch.max(data.y)+1)+500:20*(torch.max(data.y)+1)+1500] else False for i in range(len(indices))]

        data_train_mask = torch.tensor(mask_train)
        data_val_mask = torch.tensor(mask_val)
        data_test_mask = torch.tensor(mask_test)


    indices = [False for i in range(len(data.y))]
    mask_train = [True if i in shuffled_indices[:int(0.6*len(data.y))] else False for i in range(len(indices))]
    mask_val = [True if i in shuffled_indices[int(0.6*len(data.y)):int(0.8*len(data.y))] else False for i in range(len(indices))]
    mask_test = [True if i in shuffled_indices[int(0.8*len(data.y)):len(data.y)] else False for i in range(len(indices))]


    data_train_mask = torch.tensor(mask_train)
    data_val_mask = torch.tensor(mask_val)
    data_test_mask = torch.tensor(mask_test)

    best_val_acc = 0
    i=0
    for epoch in range(1, 2001):
        loss,val_loss = train(data_train_mask,data_val_mask)
        val_acc = test(data_val_mask)
        train_acc = test(data_train_mask)
        test_acc = test(data_test_mask)
        if val_acc > best_val_acc :
          best_val_acc = val_acc
          Test_acc = test_acc
          i=0
        i=i+1
        #print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Train_acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f},test_acc: {test_acc:.4f}')
        #print(i)
        if i == 100 :
          break

    T.append(time.time() - start_time)
    print(f'Test Accuracy: {Test_acc:.4f}')
    moy.append(Test_acc)
    ep.append(epoch)
print("-------------------------" )
print("mod" , model)
print("hidden_channels" , hidden_channels)
print("weight_decay" , weight_decay)
print("lr" , lr)
print("moyenne" , np.mean(moy))
print("std" , np.std(moy))
print("temps" , np.mean(T))
print((2*(np.std(moy))/(10))*100)


print("-------------------------" )



runs 0
Test Accuracy: 0.6486
runs 1
Test Accuracy: 0.8108
runs 2
Test Accuracy: 0.6757
runs 3
Test Accuracy: 0.6757
runs 4
Test Accuracy: 0.6757
runs 5
Test Accuracy: 0.6216
runs 6
Test Accuracy: 0.5676
runs 7
Test Accuracy: 0.7027
runs 8
Test Accuracy: 0.6486
runs 9
Test Accuracy: 0.7568
-------------------------
mod GCN(
  (conv1): GCNConv(1703, 32)
  (conv2): GCNConv(32, 5)
)
hidden_channels 32
weight_decay 5e-06
lr 0.005
moyenne 0.6783783783783786
std 0.06446951590203549
temps 5.5833710670471195
1.2893903180407098
-------------------------


In [ ]:
import time
import numpy as np
ramdom = True
moy = []
ep = []
T = []
p_m =[]
nb_runs = 50
hidden_channels = 48
lr = 0.005
weight_decay = 5e-5
moy = []
ep = []
for runs in range(nb_runs):
    #model = GCN(hidden_channels=hidden_channels).to(device)
    model = GAT(hidden_channels=hidden_channels).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    criterion = torch.nn.CrossEntropyLoss()
    print("runs", runs)
    start_time = time.time()
    shuffled_indices = torch.randperm(len(data.y))
    if ramdom == True :


        indices = [False for i in range(len(data.y))]
        mask_train = [True if i in shuffled_indices[:20*(torch.max(data.y)+1)] else False for i in range(len(indices))]
        mask_val = [True if i in shuffled_indices[20*(torch.max(data.y)+1):20*(torch.max(data.y)+1)+500] else False for i in range(len(indices))]
        mask_test = [True if i in shuffled_indices[20*(torch.max(data.y)+1)+500:20*(torch.max(data.y)+1)+1500] else False for i in range(len(indices))]

        data_train_mask = torch.tensor(mask_train)
        data_val_mask = torch.tensor(mask_val)
        data_test_mask = torch.tensor(mask_test)


    indices = [False for i in range(len(data.y))]
    mask_train = [True if i in shuffled_indices[:int(0.6*len(data.y))] else False for i in range(len(indices))]
    mask_val = [True if i in shuffled_indices[int(0.6*len(data.y)):int(0.8*len(data.y))] else False for i in range(len(indices))]
    mask_test = [True if i in shuffled_indices[int(0.8*len(data.y)):len(data.y)] else False for i in range(len(indices))]


    data_train_mask = torch.tensor(mask_train)
    data_val_mask = torch.tensor(mask_val)
    data_test_mask = torch.tensor(mask_test)

    best_val_acc = 0
    i=0
    for epoch in range(1, 2001):
        loss,val_loss = train(data_train_mask,data_val_mask)
        val_acc = test(data_val_mask)
        train_acc = test(data_train_mask)
        test_acc = test(data_test_mask)
        if val_acc > best_val_acc :
          best_val_acc = val_acc
          Test_acc = test_acc
          i=0
        i=i+1
        #print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Train_acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f},test_acc: {test_acc:.4f}')
        #print(i)
        if i == 100 :
          break

    T.append(time.time() - start_time)
    print(f'Test Accuracy: {Test_acc:.4f}')
    moy.append(Test_acc)
    ep.append(epoch)
print("-------------------------" )
print("mod" , model)
print("hidden_channels" , hidden_channels)
print("weight_decay" , weight_decay)
print("lr" , lr)
print("moyenne" , np.mean(moy))
print("std" , np.std(moy))
print("temps" , np.mean(T))
print((2*(np.std(moy))/(10))*100)


print("-------------------------" )


runs 0
Test Accuracy: 0.6447
runs 1
Test Accuracy: 0.6754
runs 2
Test Accuracy: 0.6776
runs 3
Test Accuracy: 0.6491
runs 4
Test Accuracy: 0.6732
runs 5
Test Accuracy: 0.6294
runs 6
Test Accuracy: 0.6908
runs 7
Test Accuracy: 0.6623
runs 8
Test Accuracy: 0.6535
runs 9
Test Accuracy: 0.6711
runs 10
Test Accuracy: 0.6447
runs 11
Test Accuracy: 0.6732
runs 12
Test Accuracy: 0.6338
runs 13
Test Accuracy: 0.6404
runs 14
Test Accuracy: 0.6776
runs 15
Test Accuracy: 0.6228
runs 16


KeyboardInterrupt: 

Compute the Spectral GAP

In [ ]:
from torch_geometric.utils import to_dense_adj
def spectral_gap(adj):
    Adj = to_dense_adj(adj.squeeze()).squeeze()
    G = nx.Graph(Adj.numpy())
    eigenvalues = nx.normalized_laplacian_spectrum(G)

    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues_sorted = eigenvalues[sorted_indices]

    spectral_gap = eigenvalues_sorted[1]
    sum_of_eigenvalues = np.sum(eigenvalues_sorted[1:])

    normalized_spectral_gap = spectral_gap / sum_of_eigenvalues
    return normalized_spectral_gap


In [ ]:
after1 = spectral_gap(two_hop_tensor)#edge_index_curvp
before = spectral_gap(data_und)
print("improvement of the spectral gap of", ((after1 - before)/before)*100, "%" )

improvement of the spectral gap of -19.70755316810613 %


In [ ]:
after = spectral_gap(edge_index_curvm)#edge_index_curvp
#before = spectral_gap(data_und)
print("improvement of the spectral gap of", ((after - before)/before)*100, "%" )

improvement of the spectral gap of 63.122031313100145 %
